In [90]:
import pandas as pd
import numpy as np
import sqlite3,os,re,jieba,pickle,joblib
from cal_tfidf import *

In [2]:
datapath = os.path.join('..','data','20190710')

In [3]:
def save_joblib(obj,name):
    with open(name+'.pkl','wb') as f:
        joblib.dump(obj,f)
def load_joblib(name):
    with open(name+'.pkl','rb') as f:
        return joblib.load(f)

In [4]:
conn = sqlite3.connect(os.path.join(datapath,'aso20190710.db'))
sqlcode_bitd = "select * from baseinfo_td"
sqlcode_ver = 'select * from version'
bs = pd.read_sql(sqlcode_bitd, conn)

In [5]:
mergepkg = pd.read_csv(os.path.join(datapath,'merpkg1zuizhong.csv'),encoding = 'gbk')

In [6]:
print(bs.shape)
print(mergepkg.shape)
print(bs.columns)
print(mergepkg.columns)

(13088, 28)
(8031, 8)
Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'type_code_TD',
       'pkgName', 'type_TD', 'qimaiId', 'rdtimestamp'],
      dtype='object')
Index(['Unnamed: 0', 'type_code', 'pkgName', 'type', 'appleid', 'qimaiId',
       'bundleId', 'appName'],
      dtype='object')


In [7]:
bs2 = bs.drop(columns = ['type_code_TD','pkgName', 'type_TD', 'qimaiId'],axis = 1)
mergepkg2 = mergepkg.drop(columns = ['qimaiId','bundleId', 'appName'],axis = 1)
mergepkg2['appleid'] = pd.to_numeric(mergepkg2['appleid'], errors='coerce')
mergepkg2 = mergepkg2.dropna(subset = ['appleid'])

In [8]:
print(bs2.shape)
print(bs2.columns)
print(mergepkg2.shape)
print(mergepkg2.columns)

(13088, 24)
Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'rdtimestamp'],
      dtype='object')
(4540, 5)
Index(['Unnamed: 0', 'type_code', 'pkgName', 'type', 'appleid'], dtype='object')


In [9]:
bs3 = bs2.copy()
bs3['appleid'] = bs3['appleid'].apply(int)
mergepkg3 = mergepkg2.copy()
mergepkg3['appleid'] = mergepkg3['appleid'].apply(int)

In [10]:
bs3pkg = bs3.merge(mergepkg3,on = 'appleid',how = 'inner')
bs3pkg = bs3pkg.drop(columns =['Unnamed: 0'])
bs3pkg = bs3pkg.reset_index(drop = True)

In [11]:
print(bs3pkg.shape)
print(bs3pkg.columns)

(4448, 27)
Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'rdtimestamp',
       'type_code', 'pkgName', 'type'],
      dtype='object')


In [12]:
bs3pkg.head(2)

,appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,...,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid,introcontent,rdtimestamp,type_code,pkgName,type
0,281736535,Enigmo,游戏,¥12.00,2017-11-17,"Pangea Software, Inc.","Pangea Software, Inc.",游戏、智力游戏、娱乐、街机游戏,2008年07月11日,2017年11月17日,...,4+,"Pangea Software, Inc.",23,PangeaVR HD|Air Wings Intergalactic|Air Wings®...,373454824|590972134|496386846|298032129|288713...,•NETWORKEDMULTIPLAYERMODEWITHGAMECENTER!•Voted...,1215705600,T200200,com.westlake.enigmo,冒险游戏
1,307751675,PopStar!消灭星星,游戏,¥6.00,2015-10-22,"Zplay (Beijing) Info. Tech. Co.,Ltd.",ZPLAY,游戏、智力游戏、桌面游戏、娱乐,2009年10月23日,2015年10月22日,...,4+,ZPLAY,80,BB弹 -全民街机打砖块|CC弹-别动我大象|弹弹猫|环形世界|PopStar! 消灭星星L...,1088720922|1107521460|1077670526|1071748533|35...,亲爱的PopStar!消灭星星的玩家，请将您对此版本的意见和所用设备反馈到我们的微信订阅号中...,1256227200,T201300,ast.game.popstar,休闲游戏


In [13]:
appleid_num = len(set(bs3pkg['appleid'].values.tolist()))
print('样本中的appleid个数为： {}'.format(appleid_num))

样本中的appleid个数为： 1675


In [14]:
appleidgroup = bs3pkg['appleid'].groupby(bs3pkg['type_code'])
table_category_num = appleidgroup.agg('nunique')
table_category_num

type_code
T200100    110
T200200     64
T200300     49
T200400     59
T200500     28
T200600     95
T200700      9
T200800     13
T200900     46
T201000      2
T201100      3
T201200     26
T201300    322
T201400     73
T201500      2
T201600    226
T201700    212
T201800     34
T201900    158
T202000    235
Name: appleid, dtype: int64

# 保存数据到database

In [15]:
conn2 = sqlite3.connect(os.path.join(datapath,'aso20190719.db'))
c2 = conn2.cursor()

In [16]:
try:
    c2.execute('''create table baseinfo_sample_1675
    (tableindex integer primary key,
    appleid integer,
    appName varchar(50),
    appType varchar(10),
    price varchar(10),
    latestVersion varchar(20),
    developerFirm varchar(50),
    developer varchar(50),
    tags varchar(50),
    releaseDate varchar(20),
    lastestDate varchar(20),
    bundleId varchar(50),
    lastestVer varchar(20),
    size varchar(10),
    payInApp varchar(10),
    support varchar(20),
    compatibility varchar(50),
    lang varchar(20),
    contentRank varchar(10),
    artistname varchar(50),
    samepubappnum integer,
    samepubapplist text,
    samepubapplistid text,
    introcontent text,
    rdtimestamp integer,
    type_code varchar(20),
    pkgName varchar(50),
    type varchar(20))''')
    conn2.commit()
    print('baseinfo_sample_1675 create')
except:
    print('baseinfo_sample_1675 exits')

baseinfo_sample_1675 exits


In [17]:
# # 存入数据库
# for r in bs3pkg.itertuples():
#     temp = tuple([i for i in r])
#     try:
#         c2.execute('insert into baseinfo_sample_1675 values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',temp)
#         conn2.commit()
#     except:
#         print('error: {}'.format(getattr(r,'appleid')))

In [18]:
# test = pd.read_sql("select * from baseinfo_sample_1675",conn2)
# print(test.shape)
# print(test.columns)

# 删除掉4个category的数据T200700, T201000, T201100, T201500

In [19]:
bs3pkg2 = bs3pkg[~bs3pkg['type_code'].isin(['T200700', 'T201000', 'T201100', 'T201500'])]
bs3pkg2 = bs3pkg2.reset_index(drop = True)

In [20]:
print(bs3pkg2.shape)
print(bs3pkg2.columns)

(4420, 27)
Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'rdtimestamp',
       'type_code', 'pkgName', 'type'],
      dtype='object')


In [21]:
try:
    c2.execute('''create table baseinfo_sample_1659
    (tableindex integer primary key,
    appleid integer,
    appName varchar(50),
    appType varchar(10),
    price varchar(10),
    latestVersion varchar(20),
    developerFirm varchar(50),
    developer varchar(50),
    tags varchar(50),
    releaseDate varchar(20),
    lastestDate varchar(20),
    bundleId varchar(50),
    lastestVer varchar(20),
    size varchar(10),
    payInApp varchar(10),
    support varchar(20),
    compatibility varchar(50),
    lang varchar(20),
    contentRank varchar(10),
    artistname varchar(50),
    samepubappnum integer,
    samepubapplist text,
    samepubapplistid text,
    introcontent text,
    rdtimestamp integer,
    type_code varchar(20),
    pkgName varchar(50),
    type varchar(20))''')
    conn2.commit()
    print('baseinfo_sample_1659 create')
except:
    print('baseinfo_sample_1659 exits')

baseinfo_sample_1659 exits


In [22]:
# # 存入数据库
# for r in bs3pkg2.itertuples():
#     temp = tuple([i for i in r])
#     try:
#         c2.execute('insert into baseinfo_sample_1659 values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',temp)
#         conn2.commit()
#     except:
#         print('error: {}'.format(getattr(r,'appleid')))

# 提取出typecode和appleid的对应关系，保存到pickle

In [23]:
type_code = list(set(bs3pkg2['type_code'].values.tolist()))
dict_type_code_id = {}
for i in type_code:
    temp = bs3pkg2[bs3pkg2['type_code'].isin([i])]
    tempappleidlist = list(set(temp['appleid'].values.tolist()))
    dict_type_code_id[i] = tempappleidlist

In [24]:
for k,v in dict_type_code_id.items():
    print(k + '：'+str(len(v)))

T201700：212
T201600：226
T200200：64
T200900：46
T200500：28
T200400：59
T200600：95
T201400：73
T202000：235
T201300：322
T200800：13
T200100：110
T200300：49
T201800：34
T201900：158
T201200：26


In [25]:
# # 存一下对应关系
# with open(os.path.join(datapath,'dict_type_id_1659.pkl'), 'wb') as f:
#     pickle.dump(dict_type_code_id,f,pickle.HIGHEST_PROTOCOL)

# 计算根据intro的tfidf相似度

In [26]:
# 读取数据
intro_appleid = load_joblib(os.path.join(datapath,'dict_intro'))

In [27]:
for k,v in intro_appleid.items():
    print(k)
    print(v)
    break

1226962741
['聚游', '湖南', '麻将', '一款', '休闲类', '棋牌', '游戏', '时尚', '简约', '棋牌', '画面', '真人', '实时', '对战', '玩法', '陪伴']


In [29]:
baselistoftextlist = []
dict_idinsim_typecode = {}
i = 0
for k in dict_type_code_id:
    dict_idinsim_typecode[k] = i
    templist = dict_type_code_id[k]
    temptextlist = []
    for j in templist:
        try:
            temptextlist += intro_appleid[str(j)]
        except:
            continue
    baselistoftextlist.append(temptextlist)
    i += 1

In [30]:
# save_joblib(baselistoftextlist,os.path.join(datapath,'listoftextlist_proto_1659'))
# save_joblib(dict_idinsim_typecode,os.path.join(datapath,'dict_idinsim_proto_1659'))

In [31]:
tfidfmodel_proto_1659 = tfidfmodel_cal(baselistoftextlist)

In [61]:
dict_id_introdiff_proto_1659 = {}
for t in dict_type_code_id:
    for i in dict_type_code_id[t]:
        try:
            textlist = intro_appleid[str(i)]
            tempsim = tfidfmodel_proto_1659.cal_tfidf(textlist,dict_idinsim_typecode[t])
            tempdiff = 1-tempsim
            dict_id_introdiff_proto_1659[i] = tempdiff
        except:
            continue

In [62]:
# # 保存一下id与战略差异的对应关系
# save_joblib(dict_id_introdiff_proto_1659,os.path.join(datapath,'dict_id_introdiff_proto_1659'))

In [63]:
df_introdiff = pd.DataFrame({'appleid':[k for k in dict_id_introdiff_proto_1659],'introdiff':[dict_id_introdiff_proto_1659[k] for k in dict_id_introsim_proto_1659]})

In [64]:
df_introdiff.shape

(1476, 2)

In [65]:
df_introdiff.head()

,appleid,introdiff
0,1097465856,0.663639
1,524509185,0.924966
2,838193156,0.843238
3,1228587014,0.859037
4,1054990344,0.865313


In [66]:
# 存到数据库
try:
    c2.execute('''create table introdiff_prototype_sample_1659
    (appleid integer primary key,
    introdiff numeric)''')
    conn2.commit()
    print('introdiff_prototype_sample_1659 create')
except:
    print('introdiff_prototype_sample_1659 exits')

introdiff_prototype_sample_1659 create


In [68]:
# for k,v in dict_id_introdiff_proto_1659.items():
#     temp = (k,v)
#     try:
#         c2.execute('insert into introdiff_prototype_sample_1659 values (?,?)',temp)
#         conn2.commit()
#     except:
#         print('error: {}'.format(k))

In [69]:
# test = pd.read_sql("select * from introdiff_prototype_sample_1659",conn2)
# print(test.shape)
# print(test.columns)
# print(test.head(5))

(1476, 2)
Index(['appleid', 'introdiff'], dtype='object')
     appleid  introdiff
0  307751675   0.820942
1  342994828   0.961500
2  343200656   0.878637
3  359917414   0.872872
4  362949845   0.930567


In [70]:
# # 存到文档
# df_introdiff.to_csv(os.path.join(datapath,'introdiff_prototype_sample_1659.csv'))

# 制作需要上传的文件

In [73]:
mergepkg_up = bs3pkg2[['appleid','pkgName']].drop_duplicates()

In [74]:
print(mergepkg_up.shape)

(4420, 2)


In [75]:
mergepkg_up.to_csv(os.path.join(datapath,'upload','mergepkg_up.csv'))

In [84]:
baseinfo_up = bs3pkg2[['appleid','price','size', 'payInApp', 'samepubappnum','rdtimestamp']].drop_duplicates()

In [85]:
print(baseinfo_up.shape)

(1661, 6)


In [86]:
baseinfo_up.head()

,appleid,price,size,payInApp,samepubappnum,rdtimestamp
0,281736535,¥12.00,23.06 MB,是,23,1215705600
1,307751675,¥6.00,12.82 MB,是,80,1256227200
2,328992396,¥6.00,57.18 MB,否,19,1256227200
3,342994828,免费,41.5 MB,否,208,1260806400
6,343200656,免费,155.18 MB,是,25,1260460800


In [97]:
def price_to_float(text):
    if '免费' in text:
        price = float(0)
    elif '¥' in text:
        price = float(text.replace('¥',""))
    else:
        price = np.nan
baseinfo_up2 = baseinfo_up.copy()
baseinfo_up2['free'] = baseinfo_up2['price'].apply(lambda x: 1 if '免费' in x else 0)
baseinfo_up2['size'] = baseinfo_up2['size'].apply(lambda x: float(x.replace(' MB',"")))
baseinfo_up2['payInApp'] = baseinfo_up2['payInApp'].apply(lambda x: 1 if x == '是' else 0)
baseinfo_up2['age20161231'] = baseinfo_up2['rdtimestamp'].apply(lambda x: 1483200000-x)

In [98]:
baseinfo_up2.head()

,appleid,price,size,payInApp,samepubappnum,rdtimestamp,free,age20161231
0,281736535,¥12.00,23.06,1,23,1215705600,0,267494400
1,307751675,¥6.00,12.82,1,80,1256227200,0,226972800
2,328992396,¥6.00,57.18,0,19,1256227200,0,226972800
3,342994828,免费,41.50,0,208,1260806400,1,222393600
6,343200656,免费,155.18,1,25,1260460800,1,222739200


In [101]:
bs_diff_up = baseinfo_up2.merge(df_introdiff,on = 'appleid',how = 'inner')

In [102]:
print(bs_diff_up.shape)
print(bs_diff_up.columns)

(1476, 9)
Index(['appleid', 'price', 'size', 'payInApp', 'samepubappnum', 'rdtimestamp',
       'free', 'age20161231', 'introdiff'],
      dtype='object')


In [103]:
bs_diff_up.head()

,appleid,price,size,payInApp,samepubappnum,rdtimestamp,free,age20161231,introdiff
0,307751675,¥6.00,12.82,1,80,1256227200,0,226972800,0.820942
1,342994828,免费,41.50,0,208,1260806400,1,222393600,0.961500
2,343200656,免费,155.18,1,25,1260460800,1,222739200,0.878637
3,359917414,免费,224.04,1,34,1268928000,1,214272000,0.872872
4,362949845,¥12.00,108.99,1,13,1271779200,0,211420800,0.930567


In [105]:
# bs_diff_up.to_csv(os.path.join(datapath,'upload','bs_diff_up.csv'))

# examplar model

In [106]:
examplar_t = '672093597,823804745,1187076120,1189889599,343200656,791532221,834878585,880047117,897447840,951557681,954751394,1000619473,1022610807,1027688889,1071360255,1088720922,1120536875,1132481770,1148934777,1170455562,1178890673,1241177378,1006605215,1096974019,434811130,462737588,510582247,543097382,788406144,870091143,879295658,941017209,1068435609,1086871800,1175272674,1175672519,1191729152,1255739625,1255783557,1277061489,1280207637,1411791697,750931425,867465887,1006546980,1021136106,1061847126,1093063463,1099554323,1117273600,1169905294,1178500082,1269408720,737651307,971513085,1000619473,1053012308,1063956349,1089336971,1119059260,1143402987,1190796440,1228515946,1234193448,1237468382,593369632,895670960,910395494,940547441,1015364140,1031897589,1033387365,1088150831,1088753863,1093063463,1094162374,1109303593,1137913384,1140819787,1170455562,1178890673,1189895734,449735650,636369522,703585308,728293409,747314884,1053475918,1190223379,1260127872,1037198513,529479190,639516529,672150402,982428268,1053012308,1088179585,1099554323,1163802135,1178890673,564540143,883393043,966951791,981413101,1073870337,1102657935,1157853911,1188651309,1210593524,479516143,638689075,823804745,913292932,1012950712,1040029515,1077258852,1078806722,1094207590,1179688896,1183701879,1188931520,910395494,820140906,1094162374,1196660667,529479190,793077082,1053012308,1104855278,1117273600,897841116,954252817,954751394,1036661603,1062546241,1089336971,1152854815'
examplar = [int(i) for i in examplar_t.split(',')]

In [126]:
# save_joblib(examplar,os.path.join(datapath,'exemplar'))
len(examplar)

137

In [137]:
baselistoftextlist2 = []
dict_idinsim_typecode2 = {}
i = 0
for k in dict_type_code_id:
    dict_idinsim_typecode2[k] = i
    templist = dict_type_code_id[k]
    temptextlist = []
    for j in templist:
        if j in examplar:
            try:
                temptextlist += intro_appleid[str(j)]
            except:
                continue
    baselistoftextlist2.append(temptextlist)
    i += 1


In [138]:
# save_joblib(baselistoftextlist2,os.path.join(datapath,'listoftextlist_exemplar_1659'))
# save_joblib(dict_idinsim_typecode2,os.path.join(datapath,'dict_idinsim_exemplar_1659'))

In [139]:
tfidfmodel_exemplar_1659 = tfidfmodel_cal(baselistoftextlist2)
dict_id_introdiff_exemplar_1659 = {}
for t in dict_type_code_id:
    for i in dict_type_code_id[t]:
        if i in examplar:
            continue
        try:
            textlist = intro_appleid[str(i)]
            tempsim = tfidfmodel_exemplar_1659.cal_tfidf(textlist,dict_idinsim_typecode2[t])
            tempdiff = 1-tempsim
            dict_id_introdiff_exemplar_1659[i] = tempdiff
        except:
            continue

In [140]:
len(dict_id_introdiff_exemplar_1659)

1356

In [141]:
# # 保存一下id与战略差异的对应关系
# save_joblib(dict_id_introdiff_exemplar_1659,os.path.join(datapath,'dict_id_introdiff_exemplar_1659'))

In [142]:
df_introdiff_exemplar = pd.DataFrame({'appleid':[k for k in dict_id_introdiff_exemplar_1659],'introdiff':[dict_id_introdiff_exemplar_1659[k] for k in dict_id_introdiff_exemplar_1659]})

In [143]:
# 存到数据库
try:
    c2.execute('''create table introdiff_exemplar_sample_1659
    (appleid integer primary key,
    introdiff numeric)''')
    conn2.commit()
    print('introdiff_exemplar_sample_1659 create')
except:
    print('introdiff_exemplar_sample_1659 exits')

introdiff_exemplar_sample_1659 exits


In [144]:
for k,v in dict_id_introdiff_exemplar_1659.items():
    temp = (k,v)
    try:
        c2.execute('insert into introdiff_exemplar_sample_1659 values (?,?)',temp)
        conn2.commit()
    except:
        print('error: {}'.format(k))

error: 838193156
error: 1054990344
error: 1156925449
error: 791111694
error: 668416040
error: 1127493677
error: 890351673
error: 919107645
error: 476553281
error: 949852244
error: 1050628187
error: 1012279413
error: 1078814844
error: 550971517
error: 1183311998
error: 1127231617
error: 1187823762
error: 992051355
error: 1214802095
error: 866945218
error: 1110646984
error: 639486670
error: 867991770
error: 1186568412
error: 522008813
error: 580765736
error: 469027061
error: 909351158
error: 868692227
error: 1049657606
error: 1191422216
error: 670873866
error: 954250967
error: 909566221
error: 1339597086
error: 1214783879
error: 1176160569
error: 824318267
error: 646640353
error: 1136667579
error: 1140037349
error: 952174948
error: 467577200
error: 491231653
error: 1264560589
error: 1108822480
error: 462199252
error: 1055699417
error: 1024000488
error: 1002265066
error: 1250378276
error: 1105855019
error: 1072159281
error: 1297871425
error: 1066748491
error: 1049234005
error: 473502311
e

In [145]:
# 存到文档
# df_introdiff_exemplar.to_csv(os.path.join(datapath,'introdiff_exemplar_sample_1659.csv'))

In [146]:
df_introdiff_exemplar.rename(columns = {'introdiff':'introdiff_e'},inplace = True)

In [147]:
bs_diff_up2 = bs_diff_up.merge(df_introdiff_exemplar,on = 'appleid',how = 'inner')

In [148]:
bs_diff_up2.to_csv(os.path.join(datapath,'upload','bs_diff_up.csv'))